In [4]:
from pybaseball import statcast_pitcher, playerid_lookup

In [14]:
# pitcher_list = [["freeland", 'kyle'], ['marquez', 'german'], ['senzatela', 'antonio'], ['feltner', 'ryan'], ['flexen', 'chris']]

In [16]:
# pitcher_lookup = playerid_lookup(pitcher_list[0][0], pitcher_list[0][1])
# pitcher_id = pitcher_lookup['key_mlbam'].iloc[0]

# start_date = "2017-01-01"

# end_date = "2024-10-21"

# data1 = statcast_pitcher(start_date, end_date, pitcher_id)

Gathering Player Data


In [18]:
pitcher_dictionary = {"Kyle Freeland": {"name": ["freeland", 'kyle'], "start_date" : "2017-01-01", "end_date":"2024-10-21"},
                       "Germán Márquez": {"name": ['marquez', 'german'],"start_date" : "2016-01-01", "end_date":"2024-10-21"},
                       "Antonio Senzatela": {"name": ['senzatela', 'antonio'], "start_date" : "2017-01-01", "end_date":"2024-10-21"},
                       "Ryan Feltner": {"name": ['feltner', 'ryan'], "start_date" : "2021-01-01", "end_date":"2024-10-21"},
                       "Chris Flexen": {"name": ['flexen', 'chris'], "start_date" : "2017-01-01", "end_date":"2024-10-21"}}

In [20]:

def get_pitcher_data(pitcher_dictionary):
    all_pitcher_data = {}

    for pitcher_name, info in pitcher_dictionary.items():
        print(f"Processing data for {pitcher_name}...")
        
        # Lookup player ID
        last_name, first_name = info['name']
        pitcher_lookup = playerid_lookup(last_name, first_name)
        
        if pitcher_lookup.empty:
            print(f"Could not find player ID for {pitcher_name}. Skipping...")
            continue
        
        pitcher_id = pitcher_lookup['key_mlbam'].iloc[0]
        
        # Get play-by-play data
        start_date = info['start_date']
        end_date = info['end_date']
        
        try:
            data = statcast_pitcher(start_date, end_date, pitcher_id)
            all_pitcher_data[pitcher_name] = data
            print(f"Successfully retrieved data for {pitcher_name}")
        except Exception as e:
            print(f"Error retrieving data for {pitcher_name}: {str(e)}")
    
    return all_pitcher_data

# Example usage
pitcher_dictionary = {
    "Kyle Freeland": {"name": ["freeland", 'kyle'], "start_date": "2017-01-01", "end_date":"2024-10-21"},
    "Germán Márquez": {"name": ['márquez', 'germán'], "start_date": "2016-01-01", "end_date":"2024-10-21"},
    "Antonio Senzatela": {"name": ['senzatela', 'antonio'], "start_date": "2017-01-01", "end_date":"2024-10-21"},
    "Ryan Feltner": {"name": ['feltner', 'ryan'], "start_date": "2021-01-01", "end_date":"2024-10-21"},
    "Chris Flexen": {"name": ['flexen', 'chris'], "start_date": "2017-01-01", "end_date":"2024-10-21"}
}

pitcher_data = get_pitcher_data(pitcher_dictionary)

# Print summary of retrieved data
for pitcher, data in pitcher_data.items():
    print(f"{pitcher}: {len(data)} rows of data")


Processing data for Kyle Freeland...
Gathering Player Data
Successfully retrieved data for Kyle Freeland
Processing data for Germán Márquez...
Gathering Player Data


c:\Users\elcoo\anaconda3\Lib\site-packages\pybaseball\utils.py:295: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))


Successfully retrieved data for Germán Márquez
Processing data for Antonio Senzatela...
Gathering Player Data
Successfully retrieved data for Antonio Senzatela
Processing data for Ryan Feltner...
Gathering Player Data
Successfully retrieved data for Ryan Feltner
Processing data for Chris Flexen...
Gathering Player Data
Successfully retrieved data for Chris Flexen
Kyle Freeland: 19163 rows of data
Germán Márquez: 17257 rows of data
Antonio Senzatela: 12258 rows of data
Ryan Feltner: 5717 rows of data
Chris Flexen: 11420 rows of data


In [21]:
import pandas as pd

In [25]:
#convert function's output from dictionary to dataframes
# change play-by-play data keys/values into dictionary for each player
player_dict_dataframes = {player: pd.DataFrame(data) for player, data in pitcher_data.items()}

#convert each player key and values to separate dataframe
freeland_df = player_dict_dataframes['Kyle Freeland']
marquez_df = player_dict_dataframes['Germán Márquez']
senzatela_df = player_dict_dataframes['Antonio Senzatela']
feltner_df = player_dict_dataframes['Ryan Feltner']
flexen_df = player_dict_dataframes['Chris Flexen']


In [33]:
flexen_df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,CH,2022-10-04,83.1,-0.58,6.80,"Flexen, Chris",622682,623167,field_out,hit_into_play,NaN,NaN,NaN,NaN,13.0,"Victor Reyes grounds out, third baseman Eugeni...",R,L,R,SEA,DET,X,5.0,ground_ball,1,1,2022,-1.23,0.85,-1.13,1.45,NaN,NaN,NaN,2,4,Top,105.95,164.95,NaN,...,11.0,82.8,1394.0,6.0,662200,623167,592200,664034,647351,553993,641487,624428,672284,571745,54.50,0.194,0.206,0.0,1.0,0.0,0.0,1.0,31,3,Changeup,3,3,3,3,3,3,3,3,Standard,Standard,208.0,0.013,-0.093,NaN,NaN
1,FF,2022-10-04,92.1,-0.63,7.04,"Flexen, Chris",622682,623167,NaN,ball,NaN,NaN,NaN,NaN,11.0,"Victor Reyes grounds out, third baseman Eugeni...",R,L,R,SEA,DET,B,NaN,NaN,0,1,2022,-0.17,1.69,-0.65,4.03,NaN,NaN,NaN,2,4,Top,NaN,NaN,NaN,...,NaN,91.2,2170.0,5.8,662200,623167,592200,664034,647351,553993,641487,624428,672284,571745,54.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,2,4-Seam Fastball,3,3,3,3,3,3,3,3,Standard,Standard,184.0,0.000,0.011,NaN,NaN
2,FF,2022-10-04,92.0,-0.51,7.00,"Flexen, Chris",622682,623167,NaN,swinging_strike,NaN,NaN,NaN,NaN,8.0,"Victor Reyes grounds out, third baseman Eugeni...",R,L,R,SEA,DET,S,NaN,NaN,0,0,2022,-0.02,1.65,0.09,2.12,NaN,NaN,NaN,2,4,Top,NaN,NaN,NaN,...,NaN,91.4,2126.0,5.9,662200,623167,592200,664034,647351,553993,641487,624428,672284,571745,54.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,1,4-Seam Fastball,3,3,3,3,3,3,3,3,Standard,Standard,193.0,0.000,-0.017,NaN,NaN
3,FC,2022-10-04,85.2,-0.46,7.03,"Flexen, Chris",679529,623167,home_run,hit_into_play,NaN,NaN,NaN,NaN,5.0,Spencer Torkelson homers (7) on a fly ball to ...,R,R,R,SEA,DET,X,NaN,fly_ball,2,1,2022,0.41,0.24,-0.14,2.44,NaN,600869.0,NaN,2,4,Top,30.96,53.96,NaN,...,24.0,84.9,2367.0,5.8,662200,623167,592200,664034,647351,553993,641487,624428,672284,571745,54.72,0.986,1.990,2.0,1.0,0.0,3.0,6.0,30,4,Cutter,3,1,1,3,3,3,3,3,Infield shift,Standard,153.0,-0.212,1.765,NaN,NaN
4,FF,2022-10-04,90.6,-0.47,7.05,"Flexen, Chris",679529,623167,NaN,ball,NaN,NaN,NaN,NaN,12.0,Spencer Torkelson homers (7) on a fly ball to ...,R,R,R,SEA,DET,B,NaN,NaN,1,1,2022,-0.11,1.54,1.24,3.34,NaN,600869.0,NaN,2,4,Top,NaN,NaN,NaN,...,NaN,89.7,2053.0,5.8,662200,623167,592200,664034,647351,553993,641487,624428,672284,571745,54.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,3,4-Seam Fastball,3,1,1,3,1,3,1,3,Infield shift,Standard,198.0,0.000,0.036,NaN,NaN


In [27]:
freeland_df.columns.tolist()

['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'description',
 'spin_dir',
 'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
 'game_type',
 'stand',
 'p_throws',
 'home_team',
 'away_team',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
 'fielder_2',
 'umpire',
 'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'pitcher.1',
 'fielder_2.1',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimat

In [28]:
freeland_df.shape

(19163, 94)

In [29]:
freeland_df.to_excel("../excel/test_data.xlsx")


### moving forward: 

1. sort by game date first
2. sort by inning second
3. add column for pitch count (running total of pitches by this player in the game)
4. add column for what was previously pitched

In [43]:
freeland_df_sorted = freeland_df.sort_values(by=['game_date', 'at_bat_number','pitch_number'])
freeland_df_sorted.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
14654,FF,2017-02-28,93.5,3.20,5.68,"Freeland, Kyle",444843,607536,NaN,ball,NaN,NaN,NaN,NaN,1.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,B,NaN,NaN,0,0,2017,0.82,1.32,-0.53,3.61,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,NaN,93.4,2300.0,6.4,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,54.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,148.0,0.000,NaN,NaN,NaN
14653,FF,2017-02-28,93.5,3.08,5.96,"Freeland, Kyle",444843,607536,NaN,foul,NaN,NaN,NaN,NaN,4.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,NaN,NaN,1,0,2017,0.46,2.43,-0.28,2.56,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,NaN,93.2,2437.0,6.7,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,53.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,169.0,0.000,NaN,NaN,NaN
14652,SL,2017-02-28,92.0,3.27,5.66,"Freeland, Kyle",444843,607536,NaN,foul,NaN,NaN,NaN,NaN,7.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,NaN,NaN,1,1,2017,0.10,1.38,-0.58,2.03,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,NaN,91.7,2283.0,6.4,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,54.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,Slider,0,0,0,0,0,0,0,0,Standard,Standard,176.0,0.000,NaN,NaN,NaN
14651,FF,2017-02-28,95.1,3.25,5.64,"Freeland, Kyle",444843,607536,NaN,foul,NaN,NaN,NaN,NaN,9.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,NaN,NaN,1,2,2017,1.27,1.58,0.50,2.16,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,NaN,94.9,2543.0,6.8,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,53.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,141.0,0.000,NaN,NaN,NaN
14650,SL,2017-02-28,87.8,3.21,5.76,"Freeland, Kyle",444843,607536,strikeout,swinging_strike,NaN,NaN,NaN,NaN,13.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,2.0,NaN,1,2,2017,-0.28,1.19,-0.97,2.69,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,NaN,87.6,2263.0,6.4,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,54.11,NaN,NaN,0.0,NaN,0.0,0.0,NaN,1,5,Slider,0,0,0,0,0,0,0,0,Standard,Standard,193.0,0.022,NaN,NaN,NaN


In [44]:
freeland_df_sorted["pitch_count"] = freeland_df_sorted.groupby("game_date").cumcount()+1
freeland_df_sorted.tail()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count
4,KC,2024-09-26,80.3,2.55,5.54,"Freeland, Kyle",502671,607536,field_out,hit_into_play,NaN,NaN,NaN,NaN,9.0,"Paul Goldschmidt grounds out, third baseman Ry...",R,R,L,COL,STL,X,5.0,ground_ball,0,2,2024,-0.14,-0.39,0.59,2.09,NaN,NaN,NaN,0,5,Top,106.12,167.58,NaN,...,81.4,2378.0,6.5,746495,607536,696100,669911,668845,641857,678662,666134,656541,595909,53.96,0.066,0.056,0.00,1.0,0.0,0.0,2.0,41,5,Knuckle Curve,5,5,5,5,5,5,5,5,Strategic,Standard,312.0,0.030,-0.146,75.62831,8.41629,81
3,SL,2024-09-26,85.6,2.65,5.65,"Freeland, Kyle",671056,607536,NaN,ball,NaN,NaN,NaN,NaN,12.0,Iván Herrera homers (5) on a fly ball to cente...,R,R,L,COL,STL,B,NaN,NaN,0,0,2024,0.04,0.44,1.02,2.95,NaN,NaN,NaN,1,5,Top,NaN,NaN,NaN,...,87.0,2263.0,6.7,746495,607536,696100,669911,668845,641857,678662,666134,656541,595909,53.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,1,Slider,5,5,5,5,5,5,5,5,Strategic,Standard,134.0,0.000,0.024,NaN,NaN,82
2,FF,2024-09-26,88.7,2.53,5.74,"Freeland, Kyle",671056,607536,NaN,ball,NaN,NaN,NaN,NaN,12.0,Iván Herrera homers (5) on a fly ball to cente...,R,R,L,COL,STL,B,NaN,NaN,1,0,2024,0.90,0.88,2.09,4.55,NaN,NaN,NaN,1,5,Top,NaN,NaN,NaN,...,89.7,2225.0,6.5,746495,607536,696100,669911,668845,641857,678662,666134,656541,595909,53.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,2,4-Seam Fastball,5,5,5,5,5,5,5,5,Strategic,Standard,138.0,0.000,0.046,NaN,NaN,83
1,CH,2024-09-26,89.4,2.41,5.80,"Freeland, Kyle",671056,607536,home_run,hit_into_play,NaN,NaN,NaN,NaN,8.0,Iván Herrera homers (5) on a fly ball to cente...,R,R,L,COL,STL,X,NaN,fly_ball,2,0,2024,0.97,0.75,-0.09,1.95,NaN,NaN,NaN,1,5,Top,82.55,23.33,NaN,...,90.7,2140.0,6.5,746495,607536,696100,669911,668845,641857,678662,666134,656541,595909,53.98,0.831,2.000,2.00,1.0,0.0,3.0,6.0,42,3,Changeup,5,5,5,5,6,5,6,5,Strategic,Standard,147.0,-0.151,0.930,NaN,NaN,84
0,SL,2024-09-26,85.5,2.68,5.68,"Freeland, Kyle",691023,607536,double,hit_into_play,NaN,NaN,NaN,NaN,5.0,"Cardinals challenged (catch or drop), call on ...",R,R,L,COL,STL,X,9.0,fly_ball,0,0,2024,0.09,0.49,0.22,3.03,NaN,NaN,NaN,1,5,Top,195.20,64.07,NaN,...,86.3,2344.0,6.5,746495,607536,696100,669911,668845,641857,678662,666134,656541,595909,53.98,0.297,0.561,1.25,1.0,1.0,1.0,5.0,43,1,Slider,5,6,6,5,6,5,6,5,Strategic,Standard,156.0,-0.045,0.414,80.41021,7.60911,85


In [45]:
freeland_df_sorted["trailing_pitch"] = freeland_df_sorted.groupby("batter")['pitch_type'].shift(1)
freeland_df_sorted.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count,trailing_pitch
14654,FF,2017-02-28,93.5,3.20,5.68,"Freeland, Kyle",444843,607536,NaN,ball,NaN,NaN,NaN,NaN,1.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,B,NaN,NaN,0,0,2017,0.82,1.32,-0.53,3.61,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,2300.0,6.4,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,54.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,148.0,0.000,NaN,NaN,NaN,1,NaN
14653,FF,2017-02-28,93.5,3.08,5.96,"Freeland, Kyle",444843,607536,NaN,foul,NaN,NaN,NaN,NaN,4.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,NaN,NaN,1,0,2017,0.46,2.43,-0.28,2.56,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,2437.0,6.7,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,53.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,169.0,0.000,NaN,NaN,NaN,2,FF
14652,SL,2017-02-28,92.0,3.27,5.66,"Freeland, Kyle",444843,607536,NaN,foul,NaN,NaN,NaN,NaN,7.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,NaN,NaN,1,1,2017,0.10,1.38,-0.58,2.03,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,2283.0,6.4,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,54.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,Slider,0,0,0,0,0,0,0,0,Standard,Standard,176.0,0.000,NaN,NaN,NaN,3,FF
14651,FF,2017-02-28,95.1,3.25,5.64,"Freeland, Kyle",444843,607536,NaN,foul,NaN,NaN,NaN,NaN,9.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,NaN,NaN,1,2,2017,1.27,1.58,0.50,2.16,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,2543.0,6.8,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,53.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,141.0,0.000,NaN,NaN,NaN,4,SL
14650,SL,2017-02-28,87.8,3.21,5.76,"Freeland, Kyle",444843,607536,strikeout,swinging_strike,NaN,NaN,NaN,NaN,13.0,Andre Ethier strikes out swinging.,S,L,L,COL,LAD,S,2.0,NaN,1,2,2017,-0.28,1.19,-0.97,2.69,NaN,NaN,NaN,0,1,Top,NaN,NaN,NaN,...,2263.0,6.4,509647,607536,608596,435622,518934,571448,596115,467827,453568,471865,54.11,NaN,NaN,0.0,NaN,0.0,0.0,NaN,1,5,Slider,0,0,0,0,0,0,0,0,Standard,Standard,193.0,0.022,NaN,NaN,NaN,5,FF
